In [1]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import pickle
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
!cat models/20192009_4/train.csv | wc -l

89835


In [3]:
!ls models/20192009_4/

train-data-10_1.csv  train-data-1_5.csv   train-data-31_6.csv
train-data-10_4.csv  train-data-1_6.csv   train-data-33_5.csv
train-data-13_1.csv  train-data-1_7.csv   train-data-34_4.csv
train-data-13_5.csv  train-data-1_8.csv   train-data-3_1.csv
train-data-15_4.csv  train-data-1_9.csv   train-data-40_2.csv
train-data-17_3.csv  train-data-20_5.csv  train-data-40_4.csv
train-data-18_2.csv  train-data-21_3.csv  train-data-5_5.csv
train-data-19_2.csv  train-data-21_4.csv  train-data-6_1.csv
train-data-1_3.csv   train-data-2_4.csv   train.csv
train-data-1_4.csv   train-data-30_3.csv


In [4]:
features = ["f" + str(i) for i in range(0,57)]
label = ["action"]

headers = features + label

df = pd.read_csv('models/20192009_4/train.csv', 
                 sep = ',', 
                 header = None,
                 names = headers)
df

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f48,f49,f50,f51,f52,f53,f54,f55,f56,action
0,232,0,5,661,-75,657,307,-67,0,1,...,432,-83,0,2,701,-164,682,396,-92,0
1,232,0,5,656,-75,652,307,-62,0,1,...,434,-80,0,2,699,-164,680,396,-90,4
2,232,0,5,651,-75,647,307,-57,0,1,...,436,-77,0,2,698,-164,678,396,-88,5
3,240,0,5,645,-67,642,307,-52,0,1,...,438,-74,0,2,694,-156,676,396,-86,1
4,240,0,5,641,-67,637,307,-47,0,1,...,440,-71,0,2,692,-156,674,396,-84,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89830,352,-2,6,404,339,220,13,370,-2,3,...,79,-44,2,1,671,3,671,349,-81,1
89831,352,-2,6,403,341,214,11,376,-2,3,...,81,-40,2,1,670,1,670,351,-80,4
89832,360,-2,6,408,351,208,9,382,-2,3,...,83,-36,2,1,669,7,669,353,-79,1
89833,360,-2,6,407,353,202,7,388,-2,3,...,85,-32,2,1,668,5,668,355,-78,4


In [5]:
print("headers length: {}, df lenght: {}".format(len(headers), len(df)))

headers length: 58, df lenght: 89835


In [6]:
df["f0"].value_counts()

256    4287
304    3629
248    3512
240    3383
264    3262
       ... 
425       3
449       2
409       2
433       1
441       1
Name: f0, Length: 161, dtype: int64

In [7]:
print("different actions count \n{}".format(df.drop_duplicates()['action'].value_counts()))

different actions count 
5    17388
0    17081
1    16388
2    14748
4    14667
3     9563
Name: action, dtype: int64


In [8]:
print("no of states without duplicates: {}".format(len(df.drop_duplicates())))

no of states without duplicates: 89835


In [9]:
y_train_all = df["action"]
X_train_all = df.drop(columns="action")

In [10]:
print("no of states without duplicates and action column: {}".format(len(X_train_all.drop_duplicates())))

no of states without duplicates and action column: 89835


In [11]:
# import pandas_profiling
# pandas_profiling.ProfileReport(df)

In [12]:
# X_train_all[["f0", "f1"]]

In [13]:
def evaluate_model(xtrain, ytrain, xtest, ytest, model):
    y_pred_test = model.predict(xtest)
    y_pred_train = model.predict(xtrain)
    test_accuracy = accuracy_score(ytest, y_pred_test)
    train_accuracy = accuracy_score(ytrain, y_pred_train)
    print('Training accuracy: {0:.2f}, evaluation accuracy: {1:.2f}'.format(train_accuracy, test_accuracy))


Basic logisting regresion

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_train_all, y_train_all, test_size=0.2)

classifier = LogisticRegression().fit(X_train, y_train)

/anaconda3/envs/letsplay/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/envs/letsplay/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [15]:
evaluate_model(X_train, y_train, X_test, y_test, classifier)

Training accuracy: 0.23, evaluation accuracy: 0.23


In [16]:
pickle.dump(classifier, open("ai_model_logit.pkl","wb"))

xgboost

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_train_all, y_train_all, 
                                                    test_size=0.2, random_state=7)

xgb_model = XGBClassifier(n_estimators=500).fit(X_train, y_train,eval_set = [[X_test, y_test]], verbose = 1)

In [18]:
evaluate_model(X_train, y_train, X_test, y_test, xgb_model)

Training accuracy: 0.36, evaluation accuracy: 0.28


In [19]:
xgb_model = XGBClassifier(n_estimators=500).fit(X_train_all, y_train_all)

In [20]:
pickle.dump(xgb_model, open("ai_model_xgb.pkl","wb"))

random forest model

In [ ]:
forest_model = RandomForestClassifier(random_state=1, n_estimators=500)
forest_model.fit(X_train, y_train)

In [ ]:
evaluate_model(X_train, y_train, X_test, y_test, forest_model)

In [ ]:
# min_max_scaler = MinMaxScaler()
# np_scaled = min_max_scaler.fit_transform(df)
# df_normalized = pd.DataFrame(np_scaled)
# df_normalized.columns = feature_cols + ['Class']

In [ ]:
# def make_crosses(X_train, X_validation, degree=2):
#   from sklearn.preprocessing import PolynomialFeatures
#   poly = PolynomialFeatures(degree=degree)
#   X_train_poly = poly.fit_transform(X_train)
#   X_validation_poly = poly.transform(X_validation)
#   return X_train_poly, X_validation_poly

In [ ]:
mlp_model = MLPClassifier(solver='lbfgs', alpha=1e-5, 
                    hidden_layer_sizes=(24, 24), random_state=1, max_iter=200000)

mlp_model.fit(X_train_all, y_train_all)

In [ ]:
evaluate_model(X_train, y_train, X_test, y_test, mlp_model)

Save models to pkl

In [ ]:
pickle.dump(forest_model, open("ai_model_forest.pkl","wb"))

In [ ]:
pickle.dump(mlp_model, open("ai_model_mlp.pkl","wb"))

In [ ]:
!ls

In [ ]:
# model_p = pickle.load(open("ai_model.pkl","rb"))
game_state_4f = (np.array([8,280,630,-1,1,674,-3,7,667,-1,2,649,-3,2,662,0,3,662,-1,4,655,-1,5,608,2,7,1,8,288,630,-1,1,672,-3,7,668,-1,2,650,-3,2,661,0,3,657,-1,4,649,-1,5,602,2,7,1,8,296,629,-1,1,670,-3,7,669,-1,2,651,-3,2,660,0,3,651,-1,4,643,-1,5,596,2,7,1,8,296,628,-1,1,665,-3,7,667,-1,2,650,-3,2,658,0,3,647,-1,4,638,-1,5,589,2,7])
                 .reshape(1, -1))



game_state_1f = (np.array([25, -410, 308, 5, 1, -343, 483, 1, 2, -417, 592, 5, 2, -184, 589, 1, 2, -222, 275, 7, 2, -93, 143, 3, -2, -128, 549, 1, -3, -121, 399, 6, -3]).reshape(1, -1))

xgb_model.predict(game_state_1f)